## Problem One: Confirmation of competition contents

* What to learn and what to predict?

* What kind of file to create and submit to Kaggle?

* What kind of index value will be used to evaluate the submissions?





What to learn and what to predict
• To predict how capable each applicant is of repaying a loan
What kind of file to create and submit to Kaggle?
• A file predicting the probability for the TARGET variable. And should contain a header of both SK_ID_CURR and TARGET
What kind of index value will be used to evaluate the submissions?
• Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target. The ROC curve is created by plotting the true positive rate (TPR) against the false positive rate (FPR) at various threshold settings. The true-positive rate is also known as sensitivity, recall or probability of detection[8] in machine learning.

## Problem Two: Learning and verification

In [1]:
import pandas as pd
import numpy as np


In [2]:
# Loading the dataset
df_train = pd.read_csv("application_train.csv")
df_test = pd.read_csv("application_test.csv")

df_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
display(df_train.info())
display(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 121 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(16)
memory usage: 45.0+ MB


None

In [4]:
display(df_train.shape)
display(df_test.shape)

(307511, 122)

(48744, 121)

* The test dataset is missing the target column because target is the variable we're trying to predict.
* We have 16 columns of data type object

In [24]:
obj_cols = df_train.columns[df_train.dtypes == "object"]
obj_cols

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object')

In [6]:
len(df_test.columns[df_test.dtypes == "object"])

16

In [7]:
(df_train.isna().sum() > 0).value_counts()

True     67
False    55
dtype: int64

In [8]:
(df_train.isnull().sum(axis=1) > 20).value_counts()

True     184207
False    123304
dtype: int64

In [23]:
# Getting the percentage of missing values in the train dataset

percent_missing_train = (df_train.isna().sum().sort_values(ascending=False) / len(df_train) * 100).round(1)
percent_missing_train[percent_missing_train > 5]

COMMONAREA_MEDI                 69.9
COMMONAREA_AVG                  69.9
COMMONAREA_MODE                 69.9
NONLIVINGAPARTMENTS_MODE        69.4
NONLIVINGAPARTMENTS_AVG         69.4
NONLIVINGAPARTMENTS_MEDI        69.4
FONDKAPREMONT_MODE              68.4
LIVINGAPARTMENTS_MODE           68.4
LIVINGAPARTMENTS_AVG            68.4
LIVINGAPARTMENTS_MEDI           68.4
FLOORSMIN_AVG                   67.8
FLOORSMIN_MODE                  67.8
FLOORSMIN_MEDI                  67.8
YEARS_BUILD_MEDI                66.5
YEARS_BUILD_MODE                66.5
YEARS_BUILD_AVG                 66.5
OWN_CAR_AGE                     66.0
LANDAREA_MEDI                   59.4
LANDAREA_MODE                   59.4
LANDAREA_AVG                    59.4
BASEMENTAREA_MEDI               58.5
BASEMENTAREA_AVG                58.5
BASEMENTAREA_MODE               58.5
EXT_SOURCE_1                    56.4
NONLIVINGAREA_MODE              55.2
NONLIVINGAREA_AVG               55.2
NONLIVINGAREA_MEDI              55.2
E

In [ ]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()

for i in obj_cols:
    df_train[i] = le.fit_transform(df_train[i])
    df_test[i] = le.fit_transform(df_test[i])

In [48]:
new = df_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
new = pd.concat([new, df_train["TARGET"]], axis=1, join="inner")
new.corr()

,0,TARGET
0,1.000000,-0.222052
TARGET,-0.222052,1.000000


In [62]:
df_train["DAYS_ENDDATE_FACT"]

KeyError: 'DAYS_ENDDATE_FACT'

## Problem Three: Estimation on test data

## Problem Four: Feature engineering